In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [11]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

In [16]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END

In [9]:
from langchain.chat_models.base import BaseChatModel
from langchain_groq import ChatGroq

llm: BaseChatModel = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile",  # or "llama2-70b-4096"
    groq_api_key=os.getenv("GROQ_API_KEY")
)

In [13]:
response = llm.invoke("Who was the first person to walk on the moon")
response.content

'The first person to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon. He was followed by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the moon during the mission.'

In [19]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}


builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [20]:
message = {"role":"user","content":"Who was the first person to walk on the moon?"}
response = graph.invoke({"messages": [message]})

In [ ]:
response.AI

{'messages': [HumanMessage(content='Who was the first person to walk on the moon?', additional_kwargs={}, response_metadata={}, id='03889212-cd04-4da3-959c-ec574f9b2006'),
  AIMessage(content='The first person to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 46, 'total_tokens': 121, 'completion_time': 0.117136789, 'prompt_time': 0.002268524, 'queue_time': 0.058625232, 'total_time': 0.119405313}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--522dfa73-e371-4740-8237-92df405bd1ba-0', usage_metadata

In [28]:
state = None
while True:
    in_messgae = input("You: ")
    if in_messgae.lower() in ["exit", "quit"]:
        break
    if state is None:
        state: State = {
            "messages" : [{"role": "user", "content": in_messgae}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_messgae})
    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)


Bot: The lead actors in the movie "The Departed" (2006) include:

1. Leonardo DiCaprio as Billy Costigan, an undercover cop.
2. Matt Damon as Colin Sullivan, a mobster infiltrating the police department.
3. Jack Nicholson as Francis "Frank" Costello, a crime boss.

All three actors played crucial roles in the film, but if I had to choose one as the "lead," I'd say it's a tie between Leonardo DiCaprio and Matt Damon, as their characters' storylines drive the plot of the movie.
Bot: Leonardo DiCaprio's birthday is November 11, 1974.
Bot: The guitarist for Led Zeppelin is Jimmy Page. He is a highly influential and iconic guitarist, known for his powerful and distinctive playing style, which has been a key element of Led Zeppelin's sound. Page is also a skilled songwriter, producer, and arranger, and has been inducted into the Rock and Roll Hall of Fame as a member of Led Zeppelin.
Bot: Jimmy Page's birthday is January 9, 1944.
